In [ ]:
!apt install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-25ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.4 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=02eec9332f49fc8cef9f590870cdf8d30314fdc4488d825bbbbb68bd4f6c7837
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.7 MB/s eta 0:00:00


In [ ]:
!pip install open_clip_torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.5 MB/s eta 0:00:00


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import timm

In [ ]:
import wandb
import torchvision as tv
import fastai
from fastai.vision.all import *
from fastai.callback.wandb import WandbCallback
import open_clip


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!unzip -qq '/content/drive/My Drive/Geolocation/50States10K.zip' -d '/content/geolocation'

In [ ]:
def fix_arizona():
    with open("/content/geolocation/Arizona/info.txt", "r") as f:
        lines = f.readlines()
        new_lines = []
        for line in lines:
            if "2011_MM600o0TQG-zhtazRWKXeA_0.jpg" in line:
                new_lines.append("2011_MM600o0TQG-zhtazRWKXeA_0.jpg\n")
                new_lines.append("33.426471 -111.594295\n")
            else:
                new_lines.append(line)
    with open("/content/geolocation/Arizona/info.txt", "w") as f:
        f.writelines(new_lines)

fix_arizona()

In [ ]:
for state in os.listdir("/content/geolocation"):
    print(state)
    if not os.path.exists(f"/content/geolocation/{state}/info.txt"):
        continue
    with open(f"/content/geolocation/{state}/info.txt", "r") as f:
        flag = False
    
        for line in f.readlines():
            if len(line.split()) > 2:
              print(line)
            

Maine
Vermont
Alabama
Hawaii
New Jersey
Pennsylvania
Virginia
Oklahoma
Tennessee
Iowa
Idaho
Minnesota
Arizona
West Virginia
Georgia
Nevada
Wyoming
North Dakota
South Carolina
Massachusetts
Connecticut
Washington
Colorado
Rhode Island
Alaska
Louisiana
Missouri
Montana
Kentucky
North Carolina
New York
Arkansas
New Mexico
Wisconsin
New Hampshire
Kansas
Oregon
Nebraska
California
Michigan
Utah
South Dakota
Florida
Mississippi
Indiana
Maryland
Delaware
Ohio
Illinois
Texas


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
config = SimpleNamespace(
    batch_size=32,
    img_size=224,
    seed=42,
    pretrained=False,
    normalize=True,
    model_name="resnet18",
    epochs=10,                                 # FIX THIS LATER
    learning_rate=2e-3,
    resize="crop",
)

In [ ]:
import torch
import torch.nn as nn

class EquirectangularDistanceLoss(nn.Module):
    def __init__(self):
        super(EquirectangularDistanceLoss, self).__init__()

    def forward(self, preds, targets):
        # Convert latitude and longitude from degrees to radians
        lat1, lon1 = torch.deg2rad(preds[:, 0]), torch.deg2rad(preds[:, 1])
        lat2, lon2 = torch.deg2rad(targets[:, 0]), torch.deg2rad(targets[:, 1])

        # Equirectangular approximation constants
        R = 6371  # Earth's mean radius in km
        x = (lon2 - lon1) * torch.cos((lat1 + lat2) / 2)
        y = lat2 - lat1

        # Compute equirectangular distance
        distance = torch.sqrt(x**2 + y**2) * R

        # Return mean distance error as the loss
        return distance.mean()


In [ ]:
def fix_path(path):
    tmp = path.strip().split('/')[-1]
    return tmp

def get_missing_labels(path):
    files = os.listdir(path)
    existing_labels = []
    # Load info.txt
    with open(os.path.join(path, "info.txt"), "r") as f:
        lines = f.readlines()
        for line in lines:
            if "jpg" in line:
                existing_labels.append(fix_path(line))
                
    missing_labels = [file for file in files if file not in existing_labels]
    return missing_labels

In [ ]:
from fastai.vision.all import *

def get_regression_dataset_state(state):
    missing_labels = get_missing_labels(f"/content/geolocation/{state}")
    def get_image_files_with_txt(path):
        return [f for f in get_image_files(path) if f.name not in missing_labels]
    
    def fix_path(path):
        tmp = path.strip().split('/')[-1]
        return tmp

    def get_regression_label(o):
        txt_file = o.parent / 'info.txt'
        lines = [fix_path(file) if "jpg" in file else file for file in txt_file.read_text().split('\n')]
        img_name = o.name
        if img_name in lines:
            idx = lines.index(img_name)
            coords = lines[idx + 1]
            x, y = map(float, coords.split())
            return tensor([x, y]).float()
        else:
            print(f"Missing label for {img_name}")
            return None


    def filter_unlabeled_images(files):
        return [f for f in files if f.name not in missing_labels]


    path = Path('/content/geolocation')
    specific_class_path = path / state
    fnames = get_image_files_with_txt(specific_class_path)

    dblock = DataBlock(blocks=(ImageBlock, RegressionBlock),
                       get_items=get_image_files_with_txt,
                       get_y=get_regression_label,
                       splitter=RandomSplitter())

    dls = dblock.dataloaders(specific_class_path)

    return dls

dls = get_regression_dataset_state('Texas')

In [ ]:
def load_data():
  resize_method = ResizeMethod.Squish if config.resize == "squish" else ResizeMethod.Crop
  # dls = get_regression_dataset_ignore_classes('/content/geolocation', 0.2, config.seed, config.batch_size, False, config.img_size, resize_method)

  mean, std = (0.48145466, 0.4578275, 0.40821073), (
      0.26862954,
      0.26130258,
      0.27577711,
  )

  if config.normalize:
      dls.add_tfms([Normalize.from_stats(mean, std)], "after_batch")

  return dls

dls = load_data()
print("Num classes =", dls.c)
print(len(dls.train_ds), len(dls.valid_ds))


Num classes = 2
8000 2000


In [ ]:
# callback = WandbCallback(log_preds=True, log_preds_every_epoch=True, log_model=True)
from fastai.callback.wandb import log_model
def new_after_epoch(self):
  "Log validation loss and custom metrics & log prediction samples"
  # Correct any epoch rounding error and overwrite value
  self._wandb_epoch = round(self._wandb_epoch)
  if self.log_preds and self.log_preds_every_epoch:
      self.log_predictions()
  wandb.log({'epoch': self._wandb_epoch}, step=self._wandb_step)
  wandb.log({n:s for n,s in zip(self.recorder.metric_names, self.recorder.log) if n not in ['train_loss', 'epoch', 'time']}, step=self._wandb_step)


  name = f"{learn.cbs[-2].fname}_epoch_{self._wandb_epoch}"
  learn.save(name, with_opt=True)
  # metadata = {n:s for n,s in zip(self.recorder.metric_names, self.recorder.log) if n not in ['train_loss', 'epoch', 'time']}
  # log_model(self.save_model.last_saved_path, name=self.save_model.fname, metadata=metadata) 
  # log_model(f"{name}.pth", name=self.save_model.fname+self._wandb_epoch, metadata=metadata)     



WandbCallback.after_epoch = new_after_epoch

In [ ]:
train_cbs = [
    WandbCallback(log_preds=True, log_preds_every_epoch=True, log_model=True),
]

other_cbs = [
]


def equirectangular_rmse(preds, targets):
    loss_func = EquirectangularDistanceLoss()
    distance = loss_func(preds, targets)
    return torch.sqrt(distance)

def equirectangular_mae(preds, targets):
    loss_func = EquirectangularDistanceLoss()
    distance = loss_func(preds, targets)
    return distance

learn = vision_learner(
    dls,
    config.model_name,
    metrics=[equirectangular_rmse, equirectangular_mae],
    concat_pool=True,
    cbs=train_cbs,
    # loss_func=EquirectangularDistanceLoss()
).to_fp16()

learn.model.to(device)
None

In [ ]:
learn.loss_func

FlattenedLoss of MSELoss()

In [ ]:
# learn.cbs[-2].fname

In [ ]:
def wandb_init():
    wandb.login()

wandb_init()

In [ ]:
"""path='/content/geolocation/models/model.pth'
path = Path(path)
if not path.is_file():
    raise f'path must be a valid file: {path}'
name = None
name = ifnone(name, f'run-{wandb.run.id}-model')
_format_metadata(metadata)    
artifact_model = wandb.Artifact(name=name, type='model', metadata=metadata, description=description)
with artifact_model.new_file(str(Path(name).with_suffix(".pth")), mode='wb') as fa:
    fa.write(path.read_bytes())
wandb.run.log_artifact(artifact_model)
"""

'path=\'/content/geolocation/models/model.pth\'\npath = Path(path)\nif not path.is_file():\n    raise f\'path must be a valid file: {path}\'\nname = None\nname = ifnone(name, f\'run-{wandb.run.id}-model\')\n_format_metadata(metadata)    \nartifact_model = wandb.Artifact(name=name, type=\'model\', metadata=metadata, description=description)\nwith artifact_model.new_file(str(Path(name).with_suffix(".pth")), mode=\'wb\') as fa:\n    fa.write(path.read_bytes())\nwandb.run.log_artifact(artifact_model)\n'

In [ ]:
with wandb.init(config=config, project="geolocation"):
    # log_model('model.pth')
    print(f"Finding optimal learning rate")
    # lrs = learn.lr_find()
    # lr = 0.002511886414140463
    # lr = lrs.valley

    print(f"Creating learner for {config.model_name}")
    # print(f"Learning rate = {lr}")
    ti = time.perf_counter()
    # learn.fine_tune(base_lr=lr, epochs=config.epochs)
    learn.unfreeze()
    learn.fit_one_cycle(config.epochs)
    wandb.summary["fit_time"] = time.perf_counter() - ti

Finding optimal learning rate
Creating learner for resnet18


epoch,train_loss,valid_loss,equirectangular_rmse,equirectangular_mae,time
0,10953.534180,10963.639648,104.707275,10963.639648,00:21
1,10631.515625,10271.625977,101.348900,10271.625977,00:23
2,8054.400391,7131.708008,84.444817,7131.708008,00:22
3,3814.772949,2164.170898,46.498631,2164.170898,00:22
4,1080.187134,296.381470,17.206034,296.381470,00:22
5,754.438721,271.126129,16.456560,271.126129,00:22
6,700.379822,299.063782,17.259924,299.063782,00:22
7,660.482483,275.092926,16.566517,275.092926,00:22
8,642.158691,268.912598,16.386358,268.912598,00:22
9,637.327332,269.476166,16.401300,269.476166,00:21


Better model found at epoch 0 with valid_loss value: 10963.6396484375.
Better model found at epoch 1 with valid_loss value: 10271.6259765625.
Better model found at epoch 2 with valid_loss value: 7131.7080078125.
Better model found at epoch 3 with valid_loss value: 2164.1708984375.
Better model found at epoch 4 with valid_loss value: 296.3814697265625.
Better model found at epoch 5 with valid_loss value: 271.1261291503906.
Better model found at epoch 8 with valid_loss value: 268.91259765625.


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
equirectangular_mae,██▅▂▁▁▁▁▁▁
equirectangular_rmse,██▆▃▁▁▁▁▁▁
lr_0,▁▁▂▃▄▅▆▇▇██████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▂▁▁▁▁▁
lr_1,▁▁▂▃▄▅▆▇▇██████▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▂▁▁▁▁▁
mom_0,██▇▇▆▄▃▂▂▁▁▁▁▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇▇█████
mom_1,██▇▇▆▄▃▂▂▁▁▁▁▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇▇█████
raw_loss,█████████▇▆▆▅▄▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
sqr_mom_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [ ]:
wandb.init()

In [ ]:
learn.show_results()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
learn.dls.c

In [ ]:
interp.plot_top_losses(10)

In [ ]:
interp.plot_confusion_matrix(figsize=(15,15), dpi=60)

In [ ]:
learn.loss_func

EquirectangularDistanceLoss()